In [1]:
#Importing necessary libraries

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
# WebScrapping

url=('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969')
html_data = requests.get(url).text
soup = BeautifulSoup(html_data,'html5lib')
toronto_table = soup.find_all('table', class_="wikitable sortable")

In [3]:
# Convert to Dataframe

df = pd.read_html(str(toronto_table))[0]
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


## Eliminate Borough=Not assigned

In [4]:
df=df[~df.Borough.str.contains("Not assigned")]
df=df.reset_index(drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
# Second way to drop not assigned

# for i in range(0,df.shape[0]):
#     if df.loc[i,'Borough']== 'Not assigned':
#         df=df.drop(i)
# df.reset_index(drop=True)        


In [6]:
### Third way to drop not assigned

# df2 = df2.set_index('Borough', drop=False)
# df2 = df2.drop('Not assigned', axis=0)
# df2 = df2.reset_index(drop=True)
# df2.head()

## Check whether there is a dublicated Postal Code

In [7]:
print('Number of dublicated Postal Code is',df['Postal Code'].duplicated().values.sum())

Number of dublicated Postal Code is 0


## Extract Location Data and Merge with Neighborhood Data

In [8]:
#Extract location Data https://cocl.us/Geospatial_data

location_data=pd.read_csv(r'C:\Users\tunah\OneDrive\Masaüstü\IBM Data Science\10)Applied DataScience Capstone\Week3-clustering\Project- segmenting and clustering neighborhoods in the city of Toronto\Geospatial_Coordinates.csv')
location_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# Merge location data and neighborhood data

toronto_data=pd.merge(df,location_data,how='left',on='Postal Code')
toronto_data.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Creating Clusters by using venues in Toronto
Data will be taken from foursquare.

In [10]:
#Drop Postal Code since we will not use it

toronto_data=toronto_data.drop("Postal Code",axis=1)
toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
# Importing Libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# import k-means from clustering stage


print('Libraries imported')

Libraries imported


In [12]:
# Extract the geograpical cordinates of Toronto

address = 'Toronto'

geolocator = Nominatim(user_agent="et_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
# Lets narrow our map to see the clusters in a better way:
toronto_data = toronto_data[toronto_data['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'TSKO3HK045HRBVCMSYJRRTNTAMFJMFILJT5EEIASZTZIFMSU' # your Foursquare ID
CLIENT_SECRET = 'VHN2YRRYMLNQH5LE02GONCFMFHKDNNJVW2YKMN15KF40TBG' # your Foursquare Secret
ACCESS_TOKEN = 'NU5OQUYBX01BFICYB5PZUMD4E1X3RUNXLLFDUYJVXXNLA12Z' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TSKO3HK045HRBVCMSYJRRTNTAMFJMFILJT5EEIASZTZIFMSU
CLIENT_SECRET:VHN2YRRYMLNQH5LE02GONCFMFHKDNNJVW2YKMN15KF40TBG


In [16]:
# Let's explore the first neighborhood in our dataframe.
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [17]:
# Get the neighborhood's latitude and longitude values.

neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [18]:
# Now, let's get the top 100 venues that are in Regent Park, Harbourfront within a radius of 500 meters.

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&oauth_token={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude,
    ACCESS_TOKEN,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=TSKO3HK045HRBVCMSYJRRTNTAMFJMFILJT5EEIASZTZIFMSU&client_secret=VHN2YRRYMLNQH5LE02GONCFMFHKDNNJVW2YKMN15KF40TBG&v=20180605&ll=43.6542599,-79.3606359&oauth_token=NU5OQUYBX01BFICYB5PZUMD4E1X3RUNXLLFDUYJVXXNLA12Z&radius=500&limit=100'

In [19]:
# Send the GET request and examine the resutls
results = requests.get(url).json()


In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
# Clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


<ipython-input-21-978314d40244>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,Body Blitz Spa East,Spa,43.654735,-79.359874


#### Explore Neighborhoods in Toronto

In [22]:
# Let's create a function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&oauth_token={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            ACCESS_TOKEN,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# Create a new dataframe called toronto_venues.

toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harb

In [24]:
# Check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()


(2191, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [25]:
# Check how many venues were returned for each neighborhood

total_venues=toronto_venues.groupby('Neighborhood').count()
total_venues=total_venues['Venue']
total_venues

Neighborhood
Berczy Park                                                                                                   100
Brockton, Parkdale Village, Exhibition Place                                                                   32
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                           23
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     19
Central Bay Street                                                                                            100
Christie                                                                                                       29
Church and Wellesley                                                                                          100
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                 

In [26]:
# Check how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


### Analyze each neighborhood

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
neighborhood_column=toronto_onehot.pop('Neighborhood')
toronto_onehot.insert(0, 'Neighborhood', neighborhood_column)

toronto_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfro

In [28]:
# Examine the new dataframe size.
toronto_onehot.shape

(2191, 272)

In [29]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.0

### Let's print each neighborhood along with the top 10 most common venues

In [30]:
# Write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
# Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Sandwich Place,Bakery,Cocktail Bar,Pub,Italian Restaurant,Bistro,Café,Seafood Restaurant,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bakery,Convenience Store,Breakfast Spot,Sandwich Place,Burrito Place,Japanese Restaurant,Bar,Stadium
2,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Fast Food Restaurant,Light Rail Station,Yoga Studio,Brewery,Restaurant,Butcher,Farmers Market,Auto Workshop,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Rental Car Location,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Plane,Sculpture Garden,Boutique,Boat or Ferry
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Italian Restaurant,Bookstore,Thai Restaurant


# Cluster Neighborhoods 

In [32]:
# import k-means from clustering stage
from sklearn.cluster import KMeans


In [33]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged=pd.merge(toronto_data, neighborhoods_venues_sorted, how='left',left_on='Neighbourhood',right_on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,"Regent Park, Harbourfront",Coffee Shop,Pub,Park,Café,Bakery,Theater,Breakfast Spot,Performing Arts Venue,Art Gallery,Beer Store
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sandwich Place,Sushi Restaurant,Fast Food Restaurant,Chinese Restaurant,Café,Pizza Place,Italian Restaurant,Pharmacy,Burrito Place
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Lingerie Store,Burger Joint
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,St. James Town,Restaurant,Café,Italian Restaurant,Coffee Shop,Breakfast Spot,Gastropub,Seafood Restaurant,Cosmetics Shop,Hotel,Movie Theater
4,East Toronto,The Beaches,43.676357,-79.293031,0,The Beaches,Park,Coffee Shop,Pizza Place,Health Food Store,Asian Restaurant,Trail,Home Service,Pub,Discount Store,Department Store


In [35]:
# drop neighbourhood since it is repeated:
toronto_merged=toronto_merged.drop('Neighbourhood',axis=1)

In [36]:
toronto_merged

,Borough,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,43.654260,-79.360636,0,"Regent Park, Harbourfront",Coffee Shop,Pub,Park,Café,Bakery,Theater,Breakfast Spot,Performing Arts Venue,Art Gallery,Beer Store
1,Downtown Toronto,43.662301,-79.389494,0,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sandwich Place,Sushi Restaurant,Fast Food Restaurant,Chinese Restaurant,Café,Pizza Place,Italian Restaurant,Pharmacy,Burrito Place
2,Downtown Toronto,43.657162,-79.378937,0,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Lingerie Store,Burger Joint
3,Downtown Toronto,43.651494,-79.375418,0,St. James Town,Restaurant,Café,Italian Restaurant,Coffee Shop,Breakfast Spot,Gastropub,Seafood Restaurant,Cosmetics Shop,Hotel,Movie Theater
4,East Toronto,43.676357,-79.293031,0,The Beaches,Park,Coffee Shop,Pizza Place,Health Food Store,Asian Restaurant,Trail,Home Service,Pub,Discount Store,Department Store
5,Downtown Toronto,43.644771,-79.373306,0,Berczy Park,Coffee Shop,Sandwich Place,Bakery,Cocktail Bar,Pub,Italian Restaurant,Bistro,Café,Seafood Restaurant,Hotel
6,Downtown Toronto,43.657952,-79.387383,0,Central Bay Street,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Italian Restaurant,Bookstore,Thai Restaurant
7,Downtown Toronto,43.669542,-79.422564,0,Christie,Grocery Store,Café,Rental Car Location,Park,Coffee Shop,Athletics & Sports,Business Service,Restaurant,Flower Shop,Baseball Field
8,Downtown Toronto,43.650571,-79.384568,0,"Richmond, Adelaide, King",Coffee Shop,Café,Hotel,Salad Place,Gym,Steakhouse,Cosmetics Shop,Asian Restaurant,American Restaurant,Thai Restaurant
9,West Toronto,43.669005,-79.442259,0,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Pet Store,Athletics & Sports,Brewery,Smoke Shop,Middle Eastern Restaurant,Café,Bar,Bank


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In [38]:
# First Cluster

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Pub,Park,Café,Bakery,Theater,Breakfast Spot,Performing Arts Venue,Art Gallery,Beer Store
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sandwich Place,Sushi Restaurant,Fast Food Restaurant,Chinese Restaurant,Café,Pizza Place,Italian Restaurant,Pharmacy,Burrito Place
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Lingerie Store,Burger Joint
3,St. James Town,Restaurant,Café,Italian Restaurant,Coffee Shop,Breakfast Spot,Gastropub,Seafood Restaurant,Cosmetics Shop,Hotel,Movie Theater
4,The Beaches,Park,Coffee Shop,Pizza Place,Health Food Store,Asian Restaurant,Trail,Home Service,Pub,Discount Store,Department Store
5,Berczy Park,Coffee Shop,Sandwich Place,Bakery,Cocktail Bar,Pub,Italian Restaurant,Bistro,Café,Seafood Restaurant,Hotel
6,Central Bay Street,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Italian Restaurant,Bookstore,Thai Restaurant
7,Christie,Grocery Store,Café,Rental Car Location,Park,Coffee Shop,Athletics & Sports,Business Service,Restaurant,Flower Shop,Baseball Field
8,"Richmond, Adelaide, King",Coffee Shop,Café,Hotel,Salad Place,Gym,Steakhouse,Cosmetics Shop,Asian Restaurant,American Restaurant,Thai Restaurant
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Pet Store,Athletics & Sports,Brewery,Smoke Shop,Middle Eastern Restaurant,Café,Bar,Bank


In [39]:
# Second Cluster

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,"Moore Park, Summerhill East",Beer Store,Park,Playground,Tennis Court,Yoga Studio,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center
34,Rosedale,Park,Playground,Trail,Yoga Studio,Dog Run,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center


In [40]:
# Third Cluster

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Runnymede, The Junction, Weston-Pellam Park, C...",Convenience Store,Grocery Store,Pizza Place,Bus Line,Auto Workshop,Brewery,Dog Run,Diner,Discount Store,Distribution Center


In [41]:
# Fourth Cluster

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,"Forest Hill North & West, Forest Hill Road Park",Park,Health & Beauty Service,Sushi Restaurant,Jewelry Store,Bus Line,Trail,Yoga Studio,Dive Bar,Diner,Discount Store


In [42]:
# Fifth Cluster

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Home Service,Pool,Health & Beauty Service,Fast Food Restaurant,Ice Cream Shop,Dive Bar,Design Studio,Dessert Shop,Diner,Discount Store
